In [1]:
import tensorflow as tf

DIM = 4096

def a_regular_function(x, y):
    r = tf.matmul(x, y)
    return r

root = tf.train.Checkpoint()
root.f = tf.function(a_regular_function)

input_data_x = tf.constant(1., shape=[DIM, DIM], dtype=tf.float32, name=None)
input_data_y = tf.constant(1., shape=[DIM, 1  ], dtype=tf.float32, name=None)
concrete_func = root.f.get_concrete_function(input_data_x, input_data_y)

converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func])
tflite_model = converter.convert()

with open('genGEMV.tflite', 'wb') as f:
  f.write(tflite_model)

In [2]:
#Dynamic Range Quantization
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model_dynamic = converter.convert()

with open('genGEMV_dynamic.tflite', 'wb') as f:
  f.write(tflite_model_dynamic)

In [3]:
#Float16 Quantization
converter.target_spec.supported_types = [tf.float16]
tflite_model_float16 = converter.convert()

with open('genGEMV_float16.tflite', 'wb') as f:
  f.write(tflite_model_float16)

In [4]:
#Integer Float Fallback Quantization
converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func])
converter.optimizations = [tf.lite.Optimize.DEFAULT]
def representative_dataset():
    for i in range(10):
        # create fake data
        fake_data_x = tf.random.normal([DIM, DIM])
        fake_data_y = tf.random.normal([DIM, 1  ])
        yield [fake_data_x, fake_data_y]
converter.representative_dataset = representative_dataset

tflite_model_integer = converter.convert()

with open('genGEMV_integer.tflite', 'wb') as f:
  f.write(tflite_model_integer)

C:\Users\Xiaojun\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tensorflow\lite\python\convert.py:887: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
